In [1]:
import pandas as pd
import numpy as np
import nltk
import gensim
import string

In [2]:
all_Data = pd.read_csv("train_with_scraping_1.csv")
all_Data.columns

Index(['Event ID', 'Title', 'Abstract', 'Is_Food', 'ab_clean', 'ab_clean_ph',
       'zero_length', 'hashtags', 'in_reply_to_status_text', 'mentions',
       'tweet_found'],
      dtype='object')

In [3]:
all_Data.loc[all_Data["Abstract"].isnull(), 'Abstract'] = all_Data[all_Data["Abstract"].isnull()]['Title']

In [4]:
all_Data.loc[all_Data['in_reply_to_status_text'].notnull(), 'Abstract'] = all_Data[all_Data['in_reply_to_status_text'].notnull()]['in_reply_to_status_text']

In [5]:
# 测试食品相关数据
all_Data = all_Data[all_Data['Event ID'] > 0]
len(all_Data)

2958

In [6]:
all_command = all_Data.Abstract.map(lambda x: x.lstrip().rstrip())
handle_all_command = []
lines = all_command.tolist()
stopwords = set(nltk.corpus.stopwords.words("english"))
table = str.maketrans("","",string.punctuation)
for line in lines:
    line = " ".join([word for word in line.split(" ") if '@' not in word and 'http' not in word and "RT" not in word])
    tokens = nltk.word_tokenize(line)
    tokens = [w.lower() for w in tokens]
    tokens = [w.translate(table) for w in tokens]
    tokens = [w for w in tokens if w.isalpha()]
    tokens = [w for w in tokens if w not in stopwords and w ]
    handle_all_command.append(tokens)
    

In [7]:
l = [len(x) for x in handle_all_command]
sorted(l)[:10]

[0, 1, 3, 3, 3, 3, 3, 3, 3, 3]

In [8]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary


dic = Dictionary(handle_all_command)

In [9]:
corpus = [dic.doc2bow(text) for text in handle_all_command]


In [10]:
lda = LdaModel(corpus=corpus, id2word=dic, num_topics=16, passes=100) 
lda.print_topics(num_topics=16, num_words=10)

[(0,
  '0.109*"nut" + 0.101*"butters" + 0.095*"listeria" + 0.087*"recalls" + 0.081*"nationwide" + 0.061*"oskri" + 0.051*"market" + 0.042*"thrive" + 0.034*"recall" + 0.034*"due"'),
 (1,
  '0.149*"food" + 0.116*"recall" + 0.096*"dog" + 0.049*"hill" + 0.043*"pet" + 0.042*"vitamin" + 0.030*"levels" + 0.028*"nutrition" + 0.025*"canned" + 0.021*"smartnews"'),
 (2,
  '0.099*"general" + 0.099*"mills" + 0.096*"flour" + 0.087*"salmonella" + 0.055*"gold" + 0.055*"medal" + 0.051*"bags" + 0.049*"recalls" + 0.033*"fears" + 0.031*"unbleached"'),
 (3,
  '0.098*"butter" + 0.064*"found" + 0.060*"peanut" + 0.046*"cause" + 0.038*"aflatoxin" + 0.025*"products" + 0.021*"reports" + 0.021*"serious" + 0.020*"consumer" + 0.019*"government"'),
 (4,
  '0.054*"de" + 0.035*"en" + 0.028*"insider" + 0.026*"la" + 0.025*"organic" + 0.017*"duraznos" + 0.016*"por" + 0.016*"nt" + 0.016*"produce" + 0.016*"cereal"'),
 (5,
  '0.100*"perdue" + 0.094*"recall" + 0.068*"health" + 0.039*"please" + 0.032*"across" + 0.030*"food" + 

In [19]:
counter = [0 for i in range(16)]
for item in lda[corpus]:
    resTopic, maxProp = 0, 0
    for tup in item:
        if tup[1] > maxProp:
            maxProp = tup[1]
            resTopic = tup[0]
    counter[resTopic] += 1
counter

[81, 199, 1035, 21, 23, 24, 449, 297, 30, 154, 49, 149, 145, 71, 111, 120]